## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,ok


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

In [24]:
# Se establece qué columnas se eliminan

col_eliminar_creditos = ['regla_pct_ingresos','situacion_vivienda_ok','tasa_interes']
col_eliminar_tarjetas = ['nivel_tarjeta']

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)

In [25]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0


Limpieza de los datos (filtrado a nivel de filas)

In [26]:
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1) 
df_creditos.head(5)


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,0,N,ok,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok,ok


In [27]:
def regla_duracion_ingresos_vivienda(row, duracion_minima, pct_max_ingresos):
    duracion = row['duracion_credito']
    pct_ingreso = row['pct_ingreso']
    situacion_vivienda = row['situacion_vivienda']

    if duracion == duracion_minima and pct_ingreso > pct_max_ingresos:
        return 'ok' if situacion_vivienda == 'PROPIA' else 'err'
    else:
        return 'ok'

DURACION_MINIMA = 2 #Se asume que la duración mínima es de 2
PCT_MAX_INGRESOS = 0.60

df_creditos['regla'] = df_creditos.apply(regla_duracion_ingresos_vivienda, axis=1, duracion_minima=DURACION_MINIMA, pct_max_ingresos=PCT_MAX_INGRESOS)
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,estado_credito,falta_pago,regla,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,0,N,ok,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,1,Y,ok,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,0,Y,ok,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,0,N,ok,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,0,N,ok,ok


In [35]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro (df_creditos): {df_creditos.shape[0]}")
temp = df_creditos[df_creditos['edad'] < 90]
temp_c = temp[temp['antiguedad_empleado'] <= 50]
temp_c2 = temp_c[temp_c['regla_pct_ingresos'] == 'ok']
temp_c3 = temp_c2[temp_c2['regla'] == 'ok']#Agregado filtro para la duración de los ingresos de la vivienda
print(f"Filas después del filtro (df_creditos): {temp_c3.shape[0]}")

print(f"Filas antes del filtro (df_tarjetas): {df_tarjetas.shape[0]}")
temp_c4 = df_tarjetas[df_tarjetas['antiguedad_cliente'] > 15]#Aplicamos filtro de fidelidad para los clientes
print(f"Filas después del filtro (df_tarjetas): {temp_c4.shape[0]}")

Filas antes del filtro (df_creditos): 10127
Filas después del filtro (df_creditos): 9765
Filas antes del filtro (df_tarjetas): 10127
Filas después del filtro (df_tarjetas): 10007


Integración de datos

In [40]:
df_integrado = pd.merge(temp_c3, temp_c4, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrado con los filtros realizado: {coincidencias}")
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Filas del dataset integrado con los filtros realizado: 9650
Cantidad de columnas del dataset integrado: 22


Transformación de atributos

In [41]:
df_integrado['estado_credito'].value_counts()

estado_credito
0    7311
1    2339
Name: count, dtype: int64

In [42]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')

# Columna: estado_civil
cambios_estado_credito = {
    0 : 'C',
    1 : 'P',
}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')
df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,...,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,operaciones_ult_12m,personas_a_cargo
0,C,C,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,...,ok,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,111.0,3.0
1,S,P,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,...,ok,45.0,SOLTERO,PASIVO,992.0,M,3421.0,DESCONOCIDO,21.0,0.0
2,D,C,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,...,ok,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,POSGRADO_COMPLETO,23.0,3.0
3,S,C,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,...,ok,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,UNIVERSITARIO_INCOMPLETO,82.0,2.0
4,C,C,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,...,ok,41.0,CASADO,ACTIVO,4369.0,F,3128.0,UNIVERSITARIO_INCOMPLETO,59.0,2.0


Exportación de metadatos